In [275]:
import datetime
import pandas as pd
import requests

In [276]:
todaysDate = datetime.datetime.now()
delta = datetime.timedelta(days=30)
newdate = todaysDate - delta
key = newdate.strftime("%Y-%b")

In [277]:
url = "https://api.msrc.microsoft.com/cvrf/v2.0/document/" + key
headers = {'Accept': 'application/json'}
response = requests.get(url, headers=headers)
json_data = response.json()

In [278]:
vulns = json_data.get('Vulnerability', "")[1:3]

In [279]:
df_vulns = pd.json_normalize(data=vulns, sep="_")

In [280]:
df_revisionHistory = pd.json_normalize(data=vulns, record_path=['RevisionHistory'], meta=["CVE"], sep="_")
df_revisionHistory = df_revisionHistory.add_prefix('RevisionHistory_')

In [281]:
df_CVSSScore = pd.json_normalize(data=vulns, record_path=['CVSSScoreSets'], meta=["CVE"], sep="_")
df_CVSSScore = df_CVSSScore.add_prefix('CVSSScoreSets_')
df_notes = pd.json_normalize(data=vulns, record_path=['Notes'], meta=["CVE"], sep="_")
df_notes = df_notes.add_prefix('Notes_')
df_ProductStatuses = pd.json_normalize(data=vulns, record_path=['ProductStatuses'], meta=["CVE"], sep="_")
df_ProductStatuses = df_ProductStatuses.add_prefix('ProductStatuses_')
df_Threats = pd.json_normalize(data=vulns, record_path=['Threats'], meta=["CVE"], sep="_")
df_Threats = df_Threats.add_prefix('Threats_')
df_Remediations = pd.json_normalize(data=vulns, record_path=['Remediations'], meta=["CVE"], sep="_").explode('ProductID')
df_Remediations = df_Remediations.add_prefix('Remediations_')

In [282]:
product_id_list_list = df_CVSSScore.CVSSScoreSets_ProductID.tolist()
product_id_list = ["; ".join(prod_id) for prod_id in product_id_list_list]
df_CVSSScore.CVSSScoreSets_ProductID = product_id_list

df_Threats.Threats_ProductID = df_Threats.Threats_ProductID.fillna('').apply(list)
threats_product_id_list_list = df_Threats.Threats_ProductID.tolist()
threats_product_id_list = ["; ".join(prod_id) for prod_id in threats_product_id_list_list]
df_Threats.Threats_ProductID = threats_product_id_list

In [283]:
df_vulns = df_vulns.merge(df_CVSSScore, left_on="CVE", right_on="CVSSScoreSets_CVE", how='left')
df_vulns.drop('CVSSScoreSets', inplace=True, axis=1)
df_vulns.shape

(50, 16)

In [284]:
df_vulns = df_vulns.merge(df_revisionHistory, left_on="CVE", right_on="RevisionHistory_CVE", how='left')
df_vulns.drop('RevisionHistory', inplace=True, axis=1)
df_vulns.shape

(50, 19)

In [285]:
df_vulns = df_vulns.merge(df_ProductStatuses, left_on="CVE", right_on="ProductStatuses_CVE", how='left')
df_vulns.drop('ProductStatuses', inplace=True, axis=1)
df_vulns.shape

(50, 21)

In [286]:
df_vulns.shape

(50, 21)

In [287]:
df_Threats.shape

(102, 5)

In [288]:
inner_join = pd.merge(left=df_vulns, right=df_Threats, left_on ="CVSSScoreSets_ProductID", right_on="Threats_ProductID")
inner_join.shape

(128, 26)

In [289]:
df_vulns.head(5)

,Notes,DiscoveryDateSpecified,ReleaseDateSpecified,CVE,Threats,Remediations,Acknowledgments,Ordinal,Title_Value,CVSSScoreSets_BaseScore,...,CVSSScoreSets_Vector,CVSSScoreSets_ProductID,CVSSScoreSets_CVE,RevisionHistory_Number,RevisionHistory_Date,RevisionHistory_Description_Value,RevisionHistory_CVE,ProductStatuses_ProductID,ProductStatuses_Type,ProductStatuses_CVE
0,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,[{'Description': {'Value': 'Remote Code Execut...,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,...,CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:C/C:H/I:H/A:H/E...,11896,CVE-2022-22009,1.0,2022-04-12T07:00:00,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009
1,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,[{'Description': {'Value': 'Remote Code Execut...,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,...,CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:C/C:H/I:H/A:H/E...,11923,CVE-2022-22009,1.0,2022-04-12T07:00:00,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009
2,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,[{'Description': {'Value': 'Remote Code Execut...,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,...,CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:C/C:H/I:H/A:H/E...,11924,CVE-2022-22009,1.0,2022-04-12T07:00:00,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009
3,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,[{'Description': {'Value': 'Remote Code Execut...,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,...,CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:C/C:H/I:H/A:H/E...,11800,CVE-2022-22009,1.0,2022-04-12T07:00:00,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009
4,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,[{'Description': {'Value': 'Remote Code Execut...,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,...,CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:C/C:H/I:H/A:H/E...,11803,CVE-2022-22009,1.0,2022-04-12T07:00:00,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009


In [290]:
df_Threats.head(5)

,Threats_ProductID,Threats_Type,Threats_DateSpecified,Threats_Description_Value,Threats_CVE
0,11896,0,False,Remote Code Execution,CVE-2022-22009
1,11923,0,False,Remote Code Execution,CVE-2022-22009
2,11924,0,False,Remote Code Execution,CVE-2022-22009
3,11800,0,False,Remote Code Execution,CVE-2022-22009
4,11803,0,False,Remote Code Execution,CVE-2022-22009


In [291]:
inner_join = pd.merge(left=df_vulns, right=df_Threats, left_on =["CVSSScoreSets_ProductID", "CVSSScoreSets_CVE"], right_on=["Threats_ProductID", "Threats_CVE"])
inner_join.drop('Threats', inplace=True, axis=1)
inner_join.shape

(100, 25)

In [292]:
df_vulns.drop('Notes', inplace=True, axis=1)
inner_join.head(5)

,Notes,DiscoveryDateSpecified,ReleaseDateSpecified,CVE,Remediations,Acknowledgments,Ordinal,Title_Value,CVSSScoreSets_BaseScore,CVSSScoreSets_TemporalScore,...,RevisionHistory_Description_Value,RevisionHistory_CVE,ProductStatuses_ProductID,ProductStatuses_Type,ProductStatuses_CVE,Threats_ProductID,Threats_Type,Threats_DateSpecified,Threats_Description_Value,Threats_CVE
0,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11896,0,False,Remote Code Execution,CVE-2022-22009
1,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11896,3,False,Important,CVE-2022-22009
2,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11923,0,False,Remote Code Execution,CVE-2022-22009
3,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11923,3,False,Important,CVE-2022-22009
4,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11924,0,False,Remote Code Execution,CVE-2022-22009


In [293]:
df_vulns['CVSSScoreSets_ProductID'].isin(df_Threats['Threats_ProductID']).value_counts()

True    50
Name: CVSSScoreSets_ProductID, dtype: int64

In [294]:
df_Remediations.shape

(162, 11)

In [295]:
df_Remediations.head(5)

,Remediations_URL,Remediations_Supercedence,Remediations_ProductID,Remediations_Type,Remediations_DateSpecified,Remediations_AffectedFiles,Remediations_SubType,Remediations_FixedBuild,Remediations_Description_Value,Remediations_RestartRequired_Value,Remediations_CVE
0,https://catalog.update.microsoft.com/v7/site/S...,5011487,11896,2,False,[],Security Update,10.0.19043.1645,5012599,Yes,CVE-2022-22009
1,https://support.microsoft.com/help/5012599,NaN,11896,5,False,[],NaN,NaN,5012599,NaN,CVE-2022-22009
1,https://support.microsoft.com/help/5012599,NaN,11800,5,False,[],NaN,NaN,5012599,NaN,CVE-2022-22009
1,https://support.microsoft.com/help/5012599,NaN,11803,5,False,[],NaN,NaN,5012599,NaN,CVE-2022-22009
1,https://support.microsoft.com/help/5012599,NaN,11931,5,False,[],NaN,NaN,5012599,NaN,CVE-2022-22009


In [296]:
inner_join.head(5)

,Notes,DiscoveryDateSpecified,ReleaseDateSpecified,CVE,Remediations,Acknowledgments,Ordinal,Title_Value,CVSSScoreSets_BaseScore,CVSSScoreSets_TemporalScore,...,RevisionHistory_Description_Value,RevisionHistory_CVE,ProductStatuses_ProductID,ProductStatuses_Type,ProductStatuses_CVE,Threats_ProductID,Threats_Type,Threats_DateSpecified,Threats_Description_Value,Threats_CVE
0,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11896,0,False,Remote Code Execution,CVE-2022-22009
1,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11896,3,False,Important,CVE-2022-22009
2,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11923,0,False,Remote Code Execution,CVE-2022-22009
3,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11923,3,False,Important,CVE-2022-22009
4,"[{'Title': 'Description', 'Type': 2, 'Ordinal'...",False,False,CVE-2022-22009,"[{'Description': {'Value': '5012599'}, 'URL': ...","[{'Name': [{'Value': '<a href=""https://twitter...",52,Windows Hyper-V Remote Code Execution Vulnerab...,7.8,6.8,...,<p>Information published.</p>\n,CVE-2022-22009,"[11896, 11923, 11924, 11800, 11803, 11926, 11931]",3,CVE-2022-22009,11924,0,False,Remote Code Execution,CVE-2022-22009


In [297]:
inner_join = pd.merge(left=inner_join,right=df_Remediations, left_on="Threats_ProductID", right_on="Remediations_ProductID")
inner_join.shape

(410, 36)

In [298]:
inner_join = pd.merge(left=inner_join, right=df_Remediations, left_on=["Threats_ProductID", "Threats_CVE"], right_on=["Remediations_ProductID", "Remediations_CVE"])
inner_join.drop('Remediations', inplace=True, axis=1)
inner_join.shape